In [2]:
import pandas as  pd
import PIL
from pdf2image import convert_from_path
import pytesseract
import nltk
from nltk import word_tokenize, pos_tag
import spacy
import re
from contractions import fix
from nltk.corpus import stopwords
import fitz 

ModuleNotFoundError: No module named 'pdf2image'

In [11]:
nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sudha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sudha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sudha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

DEFINING SPECIFIC FUNCTIONS TO CARRY OUT PREPROCESSING

In [13]:
def to_lowercase(text):
    lowercased_text = text.lower()
    return lowercased_text

def removing_stopwords(text):
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stopwords.words('english')]
    return filtered_tokens

def removing_special_characters(text):
    # Check if text is None or not a string
    if text is None or not isinstance(text, str):
        return text  # Return input unchanged

    # Use regex to remove special characters
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return cleaned_text

def lemmatization(text):
    doc = nlp(text)

    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

EXTRACTING ALL THE TEXT FROM THE RESUME


In [14]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_number in range(doc.page_count):
        page = doc[page_number]
        text += page.get_text()
    return text

path="Sudhanwa_Bokade_ML_Resume.pdf"
resume_text_original=extract_text_from_pdf(path)


EXTRACTING RELEVANT LINKS FROM A RESUME

In [17]:
def extract_links_and_emails(resume_text):
    # Define regex patterns for matching URLs and email addresses
    link_pattern = r"(https?://[^\s]+)"
    email_pattern = r"[\w\.-]+@[\w\.-]+"

    # Use re.findall to find all matches of the patterns in the text
    links = re.findall(link_pattern, resume_text)
    emails = re.findall(email_pattern, resume_text)

    return {"Links": links, "Emails": emails}

resume_text=to_lowercase(resume_text_original)
resume_text=lemmatization(resume_text)
extract_links_and_emails(resume_text)
# NOT TAKING INTO CONSIDERATION removing_special_characters SINCE IT MAY REMOVE IMPORTANT LINKS
# NOT TAKING removing_stopwords INTO CONSIDERATION BECAUSE IT MAY DECREASE THE IMPORTANCE OF THE IMPORTANT KEYWORDS

{'Links': [], 'Emails': ['sudhanwaofficial@gmail.com']}

EXTRACTING IMPORTANT SKILLS FROM RESUME

In [19]:
nlp = spacy.load('en_core_web_sm')
# noun_chunks = nlp.noun_chunks

def extract_skills(resume_text):
    nlp_text = nlp(resume_text)

    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # reading the csv file
    data = pd.read_csv("skills.csv") 
    
    # extract values
    skills = list(data.columns.values)
    
    skillset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    
    # check for bi-grams and tri-grams (example: machine learning)
    for token in nlp_text.noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in skillset])]


resume_text=to_lowercase(resume_text_original)
resume_text=removing_stopwords(resume_text)
resume_text=removing_special_characters(resume_text)
resume_text=lemmatization(' '.join(resume_text))
extract_skills(resume_text)

['Pytorch',
 'Training',
 'Healthcare',
 'Website',
 'System',
 'Github',
 'Audio',
 'Forecasting',
 'Research',
 'Java',
 'Analysis',
 'Python',
 'Health',
 'Cloud',
 'Javascript',
 'C',
 'Ai',
 'Analyze',
 'Tensorflow',
 'Certification',
 'Architecture',
 'Css',
 'Html',
 'Flask',
 'Engineering',
 'Teaching',
 'Programming',
 'C++',
 'Access',
 'Postgresql',
 'Technical',
 'Numpy',
 'Process']

EXTRACTING IMPORTANT CERTIFICATIONS

In [ ]:

def extract_certifications(resume_text, required_certifications):
    certification_pattern = r"([A-Za-z\s]+) - (\d{4})"
    certifications = re.findall(certification_pattern, resume_text)
    
    # Compare extracted certifications with required certifications
    matching_certifications = [cert for cert in certifications if cert[0] in required_certifications]
    
    return matching_certifications

# Example usage
required_certifications = ["PMP", "AWS Certified"]
result = extract_certifications(resume_text, required_certifications)
print("Matching Certifications:", result)
